In [1]:
%load_ext autoreload
%autoreload 2
from econml._ortho_learner import _crossfit

In [14]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
class Wrapper:
    def __init__(self, model):
        self._model = model
    def fit(self, X, y, W=None):
        self._model.fit(X, y)
        return self
    def predict(self, X, y, W=None):
        return self._model.predict(X)
np.random.seed(123)
X = np.random.normal(size=(5000, 3))
y = X[:, 0] + np.random.normal(size=(5000,))
folds = list(KFold(2).split(X, y))
model = Lasso(alpha=0.01)
nuisance, model_list, fitted_inds = _crossfit(Wrapper(model),
                                 folds,
                                 X, y, W=y, Z=None)
print(nuisance)
print(model_list)
fitted_inds

(array([-1.1057289 , -1.53756637, -2.4518278 , ...,  1.10628792,
       -1.82966233, -1.78227335]),)
[<__main__.Wrapper object at 0x1273b70f0>, <__main__.Wrapper object at 0x1273b77b8>]


array([   0,    1,    2, ..., 4997, 4998, 4999])

In [15]:
from econml.dml import LinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
import numpy as np
X = np.random.normal(size=(100000, 3))
y = X[:, 0] + np.random.normal(size=(100000,))
est = LinearDMLCateEstimator(model_y=LinearRegression(), model_t=LinearRegression())
est.fit(y, X[:, 0], X[:, [1]], X[:, 2:], inference='statsmodels')
print(X[:1])
print(est.effect(X[:1, [1]]))
print(est.effect_interval(X[:1, [1]]))
print(est.coef_)
print(est.coef__interval())
print(est.const_marginal_effect(X[:1, [1]]))

[[ 0.731492    0.27402924 -2.20337664]]
[1.00000239]
(array([0.99461543]), array([1.00538936]))
[ 1.00035091 -0.00127181]
(array([ 0.99514528, -0.00641798]), array([1.00555653, 0.00387436]))
[1.00000239]


In [17]:
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, MultiTaskLassoCV
import numpy as np
X = np.random.normal(size=(5000, 10))
y = X[:, 0] + np.random.normal(size=(5000,))
est = SparseLinearDMLCateEstimator(model_y=MultiTaskLassoCV(cv=3), model_t=LassoCV(cv=3),
                                   model_final=MultiTaskLassoCV(cv=3, fit_intercept=False))
dx = 8
est.fit(np.hstack([y.reshape(-1,1), y.reshape(-1,1)]), X[:, 0], X[:, 1:dx], X[:, dx:])
print(X[:1])
print(est.effect(X[:1, 1:dx]))
print(est.model_final.coef_)
print(est.const_marginal_effect(X[:1, 1:dx]))

[[ 0.0379858   0.52725655 -0.92624606 -0.0813923  -3.68794624  0.58108286
  -0.25362371 -1.84854315  0.28964067  0.24566678]]
[[0.99416019 0.99416019]]
[[0.99416019 0.         0.         0.         0.         0.
  0.         0.        ]
 [0.99416019 0.         0.         0.         0.         0.
  0.         0.        ]]
[[0.99416019 0.99416019]]
